<a href="https://colab.research.google.com/github/epiDEN29/Vigilancia-Laboratorial/blob/main/VigiLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VigiLAB Dengue

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# ler data frame
dados = pd.read_excel('/content/drive/My Drive/Vigilância Laboratorial/data.xlsx')

# Função para processar e filtrar os dados para um agravo específico
def processar_dados_agravo(dados, agravo):
    filtrados = dados.loc[dados['Agravo da Requisição'].str.contains(agravo, case=False, na=False)].copy()

    # deletando linhas com resultados vazios
    filtrados.drop(filtrados.loc[filtrados['Status Exame'] == 'Aguardando Triagem'].index, inplace=True)
    filtrados.drop(filtrados.loc[filtrados['Status Exame'] == 'Disponível para encaminhar'].index, inplace=True)
    filtrados.drop(filtrados.loc[filtrados['Status Exame'] == 'Exame Cancelado'].index, inplace=True)
    filtrados.drop(filtrados.loc[filtrados['Status Exame'] == 'Exame em Análise'].index, inplace=True)
    filtrados.drop(filtrados.loc[filtrados['Status Exame'] == 'Exame não-realizado'].index, inplace=True)
    filtrados.drop(filtrados.loc[filtrados['1º Campo Resultado'] == 'Diagnóstico Diferencial: Não'].index, inplace=True)

    # retirando espaços das strings de resultados
    filtrados['1º Campo Resultado'] = filtrados['1º Campo Resultado'].str.strip()

    # Contar os resultados e colocá-los em colunas
    contagem_resultados = filtrados.pivot_table(index='Municipio do Solicitante',
                                                columns='1º Campo Resultado',
                                                values='Requisição',
                                                aggfunc='count',
                                                fill_value=0)

    # somando número de exames concluídos, conforme número de resultados entregues
    contagem_resultados['Amostras Enviadas'] = contagem_resultados.sum(axis=1)

    # Cria uma nova coluna 'Total Positivas' que é a soma das colunas 'Reagente' e 'Detectável'
    contagem_resultados['Total Positivas'] = contagem_resultados.get('Resultado: Reagente', 0) + \
                                             contagem_resultados.get('Dengue: Detectável', 0) + \
                                             contagem_resultados.get('Resultado: Detectável', 0)

    # Adiciona uma linha 'Total' que é a soma dos valores de cada coluna individualmente
    contagem_resultados.loc['Total'] = contagem_resultados.sum(numeric_only=True)

    # Organize as colunas de acordo com o valor total, do maior para o menor
    contagem_resultados = contagem_resultados[contagem_resultados.loc['Total'].sort_values(ascending=False).index]

    # Define colunas na ordem desejada
    ordered_columns = ['Amostras Enviadas',
                       'Dengue: Inconclusivo', 'Resultado: Inconclusivo', 'Resultado: Indeterminado',
                       'Resultado: Reagente', 'Resultado: Não Reagente',
                       'Dengue: Detectável', 'Resultado: Detectável',
                       'Dengue: Não Detectável', 'Resultado: Não Detectável',
                       'Total Positivas']

    contagem_resultados = contagem_resultados.reindex(columns=ordered_columns)

    return contagem_resultados

# Processando dados para cada agravo
df_dengue = processar_dados_agravo(dados, 'DENGUE')
df_zika = processar_dados_agravo(dados, 'ZIKA')
df_chikungunya = processar_dados_agravo(dados, 'CHIKUNGUNYA')

# Salvando os resultados em diferentes sheets no mesmo arquivo Excel
output_path = '/content/drive/My Drive/Vigilância Laboratorial/VigiLab_Pará_Arboviroses.xlsx'

with pd.ExcelWriter(output_path) as writer:
    df_dengue.to_excel(writer, sheet_name='DENGUE')
    df_zika.to_excel(writer, sheet_name='ZIKA')
    df_chikungunya.to_excel(writer, sheet_name='CHIKUNGUNYA')

print(f"Tabela salva com sucesso!")
print(f"Desenvolvido por Pedro Araújo")

## VigiLAB Oropouche

In [ ]:
# Importar bibliotecas
import pandas as pd

# Ler planilhas
data1 = pd.read_excel('C:/Users/020225/OneDrive/TRABALHOS PYTHON/ANALISE_LACEN/OROPOUCHE/data1.xlsx')
data2 = pd.read_excel('C:/Users/020225/OneDrive/TRABALHOS PYTHON/ANALISE_LACEN/OROPOUCHE/data2.xlsx')

# Concatenar os dados
dados = pd.concat([data1, data2], ignore_index=True)

# deletando linhas com resultados vazios
dados.drop(dados.loc[dados['Resultado'] == ' '].index, inplace=True)

# retirando espaços das strings de resultados
dados['Resultado'] = dados['Resultado'].str.strip()

# contando os resultados e os colocando em colunas
contagem_resultados = dados[['Mun. Requisitante',
                             'Resultado',
                             'Requisição']].groupby(by=['Mun. Requisitante', 'Resultado']).count().unstack(level=1).droplevel(0, axis=1)

# somando número de exames concluídos, conforme número de resultados entregues
contagem_resultados['Amostras Enviadas'] = contagem_resultados.sum(axis=1)

# unindo contagem_resultados e amostras_entregues em apenas um dataframe
df = pd.concat([contagem_resultados], axis=1)

# Preenche os valores ausentes nas colunas 'Reagente' e 'Detectável' com 0
df[['Reagente', 'Detectável']] = df[['Reagente', 'Detectável']].fillna(0)

# Cria uma nova coluna 'Soma' que é a soma das colunas 'Reagente' e 'Detectável'
df['Total Positivas'] = df['Reagente'] + df['Detectável']

# Adiciona uma linha 'Total' que é a soma dos valores de cada coluna individualmente
df.loc['Total'] = df.sum(numeric_only=True)

# Organize as colunas de acordo com o valor total, do maior para o menor
df = df[df.loc['Total'].sort_values(ascending=False).index]

# colocando colunas na ordem desejada e gerando arquivo em excel
df[['Amostras Enviadas',
    'Inconclusivo',
    'Indeterminado',
    'Reagente',
    'Não Reagente',
    'Detectável',
    'Não Detectável',
    'Total Positivas']].to_excel('C:/Users/020225/OneDrive/TRABALHOS PYTHON/ANALISE_LACEN/OROPOUCHE/VigiLab_Pará_Oropouche.xlsx')

print(f"Tabela salva com sucesso!")
print(f"Desenvolvido por Pedro Araújo")